In [60]:
# Bit of a mess rn but old structure is commented out and still there
#tried selenium to button click and tried to change the 'data-records-per-page' to 60, but doesn't add more to list
import os
import time
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

"""
driver = webdriver.Chrome()
url = 'https://jobs.cdc.gov/search-jobs'
driver.get(url)
button = driver.find_element_by_class_name("pagination-show-all") 
button.click()
window_after = driver.window_handles[0]
print(driver.switch_to.window(window_after).page_source)
"""
driver = webdriver.Chrome()
driver.get(url)

# Find "Cite" link by looking for anchors that contain "Cite" - second link selected "[1]"
button.driver.find_element_by_class_name("pagination-show-all") 
button.click()

print("Waiting for page to load...")
time.sleep(2) # Sleep for 2 seconds

# Get Page source after waiting for 2 seconds of current page in Chrome
source = driver.page_source

# We are done with the driver so quit.
driver.quit()

# Use BeautifulSoup to parse the html source and use "html.parser" as the Parser
soupify = bs(source, 'html.parser')


# Write the decoded data to a file named scholar.bib
with open("scholar.bib","w") as file:
    file.writelines(bibtex_html)

with requests.Session() as s:
    
        
    """
    browser = webdriver.Chrome()
    url = ('https://jobs.cdc.gov/search-jobs')
    browser.get(url)
    respData = browser.page_source
    browser.quit()
    soup = bs(respData, 'html.parser')
    print(soup)
    
    r = s.get(url)
    src = r.content
    soup = bs(src)
    soup.prettify()
    
    

    allPages = []
    data = soup.find_all('a', class_ = 'locationclick')
    for link in data:
        url = link.get('href')
        if url not in allPages:
            allPages.append(url)
    #print('Searching pages:')
    print(len(allPages))
    """

<html class="js js no-touch video no-generatedcontent audio placeholder" lang="en"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Search for available job openings at CDC" name="description"/>
<meta content="telephone=no" name="format-detection"/>
<title>Search our Job Opportunities at CDC</title>
<link href="//tbcdn.talentbrew.com/company/250/css/17896-Full.css" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,600,700,800&amp;display=swap" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Merriweather:300,400,700,900&amp;display=swap" rel="stylesheet"/>
<link href="//tbcdn.talentbrew.com/company/250/v1_0/fonts/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<link href="https://tbcdn.talentbrew.com/company/250/v3_0/js/slick/slick.css" rel="stylesheet" type="text/css"/>
<link href="https://tbcdn.talentbrew.com/company/250/v3_0/js/fancybox/jquery-fanc

In [6]:
import requests
from bs4 import BeautifulSoup as bs
import re
 
import pandas as pd

url = 'https://jobs.cdc.gov/search-jobs'

with requests.Session() as s:
    r = s.get(url)
    src = r.content
    soup = bs(src)
    soup.prettify()
    
    allPages = []
    data = soup.find_all('a', class_ = 'locationclick')
    for link in data:
        url = link.get('href')
        if url not in allPages:
            allPages.append(url)
    #print('Searching pages:')
    print(len(allPages))
    
    page_url = [] 
    job_position = [] 
    description = [] 
    location = [] 
    requirements = []
    Immunization = []
    Economics = []
    
    root_url= 'https://jobs.cdc.gov'
    
    for page in allPages:
        result = s.get(root_url + page) 
        page_source = result.content 
        soup = bs(page_source) 
        soup.prettify()
        
        for script in soup(['script','style']): 
            script.decompose()
        strips = list(soup.stripped_strings)
        strips = str(strips)
            
        immunization = ['Immunization', 'immunisation', 'vaccine', 'vaccines','vaccine-preventable diseases', 'vpd outbreak',
            'immunization campaign', 'SIA','supplemental immunization act ivities', 'cold chain', 'GAVI','shigella', 'cholera',
            'bcg', 'dtp', 'dpt', 'measles', 'influenza', 'conjugate vaccine']

        economics = ['Economics','expenditure tracking', 'financing', 
            'value for vaccination' , 'costing', 'economic analysis','costs' , 'equity', 'cost effectiveness', 'cost-effectiveness', 
            'cost benefit analysis', 'benefit-cost analysis','cost utility analysis','budget impact analysis' , 'budget' , 'budgeting' , 
            'GAVI','funding gap','fiscal']

        imm_result = any(ele in strips for ele in immunization)
        ec_result = any(ele in strips for ele in economics)

        if imm_result or ec_result:
            if imm_result: Immunization.append('True')
            else: Immunization.append('False')
            if ec_result: Economics.append('True')
            else: Economics.append('False')
                
            if (root_url+page)=='https://jobs.cdc.gov/':
                page_url = page_url
            else: 
                page_url.append(root_url+page)
            #print(page_url)

            job_data = soup.find_all('main', id = 'content')
            for data in job_data:
                div = data.find('div', class_ = 'wrapper-small')
                section = div.find('section', class_ = 'job-description')
                h1 = section.find('h1')
                string = str(h1)
                start = string.find('<h1>') + len('<h1>')
                end = string.find('</h1>')
                job = string[start:end]
                job_position.append(job)
            #print(job_position)

            description_data = soup.find_all('main', id = 'content')
            for data in description_data:
                div = data.find('div', class_ = 'wrapper-small')
                section = div.find('section', class_ = 'job-description')
                div1 = section.find('div', class_ = 'ats-description')
                div2 = str(div1)
                start = div2.find('JOB SUMMARY: </b><br/>') + len('JOB SUMMARY: </b><br/>')
                end = div2.find('<br/><br/><b>DUTIES:')
                text = div2[start:end]
                description.append(text)
            #print(description)

            requirements_data = soup.find_all('main', id = 'content')
            for data in description_data:
                div = data.find('div', class_ = 'wrapper-small')
                section = div.find('section', class_ = 'job-description')
                div1 = section.find('div', class_ = 'ats-description')
                div2 = str(div1)
                start = div2.find('DUTIES: </b><br/>[<br/> ') + len('DUTIES: </b><br/>[<br/> ')
                #print(start)
                end = div2.find(']<br/><br/><b>REQUIREMENTS:')
                #print(end)
                text = div2[start:end]
                print(text)
                requirements.append(text)
            #print(requirements)

            location_data = soup.find_all('main', id = 'content')
            for data in location_data:
                div = data.find('div', class_ = 'wrapper-small')
                span = data.find('span', class_= 'job-location job-info')
                text = span.get_text()
                location.append(text)
            #print(location)
               
           
DataFrame = pd.DataFrame() 
DataFrame['Page Url']= page_url 
DataFrame['Job']= job_position 
DataFrame['Description'] = description 
DataFrame['Requirements']= requirements 
DataFrame['Location'] = location
DataFrame['Immunization'] = Immunization
DataFrame['Economics'] = Economics

Data = DataFrame.drop_duplicates() 
#Data.to_csv("CDC_Data.csv")

print('Webscraping complete')

15
tion"><b>Department: </b>Department of Health And Human Services<br/><b>Agency: </b>Centers for Disease Control and Prevention<br/><b>Job Announcement Number: </b>HHS-CDC-M5-21-10928396<br/><b>SALARY RANGE: </b>$113581.0 to $147652.0/Per Hour<br/><b>OPEN PERIOD: </b>2020-10-14 to 2020-10-23<br/><b>SERIES &amp; GRADE: </b>GS--14/14<br/><br/><b>AGENCY MARKETING STATEMENT: </b><br/>The Centers for Disease Control and Prevention (CDC) is the agency Americans trust with their lives. As a global leader in public health, CDC is the nation's premier health promotion, prevention, and preparedness agency. Whether we are protecting the American people from public health threats, researching emerging diseases, or mobilizing public health programs with our domestic and international partners, we rely on our employees to make a real difference in the health and well-being of people here and around the world. This position is located in the U.S. Department of Health and Human Services (DHHS), Cent

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
 
import pandas as pd

url = 'https://jobs.cdc.gov/search-jobs'

with requests.Session() as s:
    r = s.get(url)
    src = r.content
    soup = bs(src)
    soup.preetify
    
    allPages = []
    allPages.append('/')
    data = soup.find_all('a', class_ = 'locationclick')
    for link in data:
        url = link.get('href')
        if url not in allPages:
            allPages.append(url)
    #print('Searching pages:')
    #print(allPages)
    
    page_url = [] 
    job_position = [] 
    description = [] 
    location = [] 
    job_department = []
    requirements = []
    Immunization = []
    Economics = []
    
    root_url= 'https://jobs.cdc.gov/'
    
    for page in allPages:
        result = s.get(root_url + page) 
        page_source = result.content 
        soup = bs(page_source) 
        soup.preetify
        
        for script in soup(['script','style']): 
            script.decompose()
            strips = list(soup.stripped_strings)
            strips = str(strips)
            #print(strips)
            
            keywords = ['Immunization', 'immunisation', 'vaccine', 'vaccines','vaccine-preventable diseases', 'vpd outbreak',
                'immunization campaign', 'SIA','supplemental immunization act ivities', 'cold chain', 'GAVI','shigella', 'cholera',
                'bcg', 'dtp', 'dpt', 'measles', 'influenza', 'conjugate vaccine', 'Economics','expenditure tracking', 'financing', 
                'value for vaccination' , 'costing', 'economic analysis','costs' , 'equity', 'cost effectiveness', 'cost-effectiveness', 
                'cost benefit analysis', 'benefit-cost analysis','cost utility analysis','budget impact analysis' , 'budget' , 'budgeting' , 
                'GAVI','funding gap','fiscal']
            
            immunization = ['Immunization', 'immunisation', 'vaccine', 'vaccines','vaccine-preventable diseases', 'vpd outbreak',
                'immunization campaign', 'SIA','supplemental immunization act ivities', 'cold chain', 'GAVI','shigella', 'cholera',
                'bcg', 'dtp', 'dpt', 'measles', 'influenza', 'conjugate vaccine']
            
            economics = ['Economics','expenditure tracking', 'financing', 
                'value for vaccination' , 'costing', 'economic analysis','costs' , 'equity', 'cost effectiveness', 'cost-effectiveness', 
                'cost benefit analysis', 'benefit-cost analysis','cost utility analysis','budget impact analysis' , 'budget' , 'budgeting' , 
                'GAVI','funding gap','fiscal']
            
            result = any(ele in strips for ele in keywords)
            #print (result) 
            
            if result == True:
                
                if (any(ele in strips for ele in immunization) == True) and (any(ele in strips for ele in economics)== False):
                    Immunization.append('True')
                    Economics.append('False')
                
                if (any(ele in strips for ele in immunization) == False) and (any(ele in strips for ele in economics)== True):
                    Immunization.append('False')
                    Economics.append('True')
                    
                if (any(ele in strips for ele in immunization) == True) and (any(ele in strips for ele in economics)== True):
                    Immunization.append('True')
                    Economics.append('True')
                
                if (root_url+page)=='https://jobs.cdc.gov/':
                    page_url = page_url
                else: 
                    page_url.append(root_url+page)
                #print(page_url)
                
                job_data = soup.find_all('main', id = 'content')
                for data in job_data:
                    div = data.find('div', class_ = 'wrapper-small')
                    section = div.find('section', class_ = 'job-description')
                    h1 = section.find('h1')
                    string = str(h1)
                    start = string.find('<h1>') + len('<h1>')
                    end = string.find('</h1>')
                    job = string[start:end]
                    job_position.append(job)
                #print(job_position)
                
                description_data = soup.find_all('main', id = 'content')
                for data in description_data:
                    div = data.find('div', class_ = 'wrapper-small')
                    section = div.find('section', class_ = 'job-description')
                    div1 = section.find('div', class_ = 'ats-description')
                    div2 = str(div1)
                    start = div2.find('JOB SUMMARY: </b><br/>') + len('JOB SUMMARY: </b><br/>')
                    end = div2.find('<br/><br/><b>DUTIES:')
                    text = div2[start:end]
                    description.append(text)
                #print(description)
                
                requirements_data = soup.find_all('main', id = 'content')
                for data in description_data:
                    div = data.find('div', class_ = 'wrapper-small')
                    section = div.find('section', class_ = 'job-description')
                    div1 = section.find('div', class_ = 'ats-description')
                    div2 = str(div1)
                    start = div2.find('DUTIES: </b><br/>[<br/> ') + len('DUTIES: </b><br/>[<br/> ')
                    #print(start)
                    end = div2.find(']<br/><br/><b>REQUIREMENTS:')
                    #print(end)
                    text = div2[start:end]
                    requirements.append(text)
                #print(requirements)
                
                location_data = soup.find_all('main', id = 'content')
                for data in location_data:
                    div = data.find('div', class_ = 'wrapper-small')
                    span = data.find('span', class_= 'job-location job-info')
                    text = span.get_text()
                    location.append(text)
                #print(location)
               
           
DataFrame = pd.DataFrame() 
DataFrame['Page Url']= page_url 
DataFrame['Job']= job_position 
DataFrame['Description'] = description 
DataFrame['Requirements']= requirements 
DataFrame['Location'] = location
DataFrame['Immunization'] = Immunization
DataFrame['Economics'] = Economics

Data = DataFrame.drop_duplicates() 
Data.to_csv("CDC_Data.csv")

print('Webscraping complete')

Webscraping complete
